계층에서의 국소적 미분값을 계산하여 입력계층쪽으로 전파한다.

In [ ]:
# 덧셈 계층은 x, y가 서로 미분에 독립적이므로, 
# 역전파 시 미분값(dout)을 그대로 역전파.
class AddLayer:
    def __init__(self):
        pass
    
    def forward(self, x, y):
        return x + y
    
    def backward(self, dout):
        dx = dout * 1
        dy = dout * 1
        return dx, dy

# 곱셈 계층은 x, y가 서로 미분에 의존적이므로,
# 역전파 시 각각 상대편의 값을 곱해서 역전파.
class MulLayer:
    def __init__(self):
        self.x = None
        self.y = None
    
    def forward(self, x, y):
        self.x = x
        self.y = y
        return x * y
    
    def backward(self, dout):
        dx = dout * self.y
        dy = dout * self.x
        return dx, dy

# ReLU 계층은 순전파 시 0 이하인 부분은 출력에 영향을 주지 않으므로,
# 역전파 시 마스킹된 부분의 미분값을 0으로 설정
class Relu:
    def __init__(self):
        self.mask = None

    def forward(self, x):
        self.mask = (x <= 0)
        out = x.copy()
        out[self.mask] = 0

        return out
    
    def backward(self, dout):
        dout[self.mask] = 0
        dx = dout

        return dx


import numpy as np

# Sigmoid 계층은 순전파 시 출력값을 저장해두고,
# 역전파 시 출력값을 이용해 미분값을 계산
class Sigmoid:
    def __init__(self):
        self.t1 = None
        self.t2 = None
        self.t3 = None

        self.out = None
    
    def forward(self, x):
        self.t1 = -x
        self.t2 = np.exp(self.t1)
        self.t3 = 1 + self.t2
        out1 = 1 / self.t3
        out2 = 1 / (1 + np.exp(-x))
        self.out = out2
        return out1, out2

    def backward(self, dout):
        # 역전파
        dy_dt3 = -1 / (self.t3 ** 2)
        dt3_dt2 = 1
        dt2_dt1 = np.exp(self.t1)
        dt1_dx = -1

        # 각 단계별 계산값을 저장하여 연쇄 미분
        dx1 = dout * dy_dt3 * dt3_dt2 * dt2_dt1 * dt1_dx
        # 출력값만 저장해서 계산 (위 방법을 정리한 식)
        dx2 = dout * (1.0 - self.out) * self.out
        return dx1, dx2

# 테스트
sigmoid = Sigmoid()

x = np.array([0.5, 1.0, -0.5])
y1, y2 = sigmoid.forward(x)
print(f"순전파 출력: {y1}, {y2}")

dout = np.array([1.0, 1.0, 1.0])

print("\n역전파 비교:")
dx1, dx2 = sigmoid.backward(dout)
print(f"역전파 출력: {dx1}, {dx2}")

print(f"\n결과가 같은가? {np.allclose(dx1, dx2)}")

순전파 출력: [0.62245933 0.73105858 0.37754067], [0.62245933 0.73105858 0.37754067]

역전파 비교:
역전파 출력: [0.23500371 0.19661193 0.23500371], [0.23500371 0.19661193 0.23500371]

결과가 같은가? True


In [ ]:
# 행렬곱 계층
class Affine:
    def __init__(self, W, b):
        self.W = W # 학습할 파라미터
        self.b = b # 학습할 파라미터
        self.x = None
        self.dW = None # 학습할 파라미터의 미분
        self.db = None # 학습할 파라미터의 미분
        

    def forward(self, x):
        self.x = x
        out = np.dot(x, self.W) + self.b
        return out

    def backward(self, dout):
        dx = np.dot(dout, self.W.T)
        self.dW = np.dot(self.x.T, dout)
        self.db = np.sum(dout, axis=0)
        return dx

행렬곱 계층의 미분계산  
L은 최종 계산된 손실값
ex) dW 계산 (손실의 W에 대한 편미분)
- `dL/dW = dL/dy * dy/dW`
- dL/dW: W가 최종 손실에 주는 영향도
- dL/dy: 현재 계층의 출력값이 최종 손실에 주는 영향도
- dy/dW: W가 현재 계층의 출력에 주는 영향도

In [ ]:
from common.functions import softmax, cross_entropy_error
class SoftmaxWithLoss:
    def __init__(self):
        self.loss = None
        self.y = None  # softmax의 출력
        self.t = None  # 정답 레이블(원-핫)

    def forward(self, x, t):
        self.t = t
        self.y = softmax(x)
        self.loss = cross_entropy_error(self.y, self.t)
        return self.loss
    
    def backward(self, dout=1):
        batch_size = self.t.shape[0]
        dx = (self.y - self.t) / batch_size

        return dx